In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import requests
import json
from datetime import datetime, timedelta
import time
import math
from collections import Counter

In [2]:
file_name = "airlines.csv"
airlines_data = pd.read_csv(file_name)
airlines_data.head()

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [3]:
file_name = "flights.csv"
flights_data = pd.read_csv(file_name, dtype={ 'YEAR': int, 'MONTH': int, 'DAY': int, 'DAY_OF_WEEK': int,
                                             'AIRLINE': np.string_, 'FLIGHT_NUMBER': int, 'TAIL_NUMBER': np.string_, 
                                             'ORIGIN_AIRPORT': np.string_, 'DESTINATION_AIRPORT': np.string_, 
                                             'SCHEDULED_DEPARTURE': int, 'DEPARTURE_TIME': float,
                                             'DEPARTURE_DELAY': float, 'TAXI_OUT': float, 'WHEELS_OFF': float,
                                             'SCHEDULED_TIME': float, 'ELAPSED_TIME': float, 'AIR_TIME': float,
                                             'DISTANCE': int, 'WHEELS_ON': float, 'TAXI_IN': float, 'SCHEDULED_ARRIVAL': int,
                                             'ARRIVAL_TIME': float, 'ARRIVAL_DELAY': float, 'DIVERTED': int, 'CANCELLED': int,
                                             'CANCELLATION_REASON': np.string_, 'AIR_SYSTEM_DELAY': float, 'SECURITY_DELAY': float, 
                                             'AIRLINE_DELAY': float,'LATE_AIRCRAFT_DELAY': float,'WEATHER_DELAY': float,})
flights_data = flights_data.drop('FLIGHT_NUMBER', 1)
flights_data = flights_data.drop('TAIL_NUMBER', 1)
flights_data = flights_data.drop('YEAR', 1)
flights_data = flights_data.drop('MONTH', 1)
flights_data = flights_data.drop('DAY', 1)
flights_data = flights_data.drop('DAY_OF_WEEK', 1)
flights_data.head()

,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,AA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,AA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
flights_data = pd.merge(flights_data, airlines_data, left_on='AIRLINE', right_on='IATA_CODE', how='left')
flights_data.drop('IATA_CODE', axis=1, inplace=True)
flights_data.rename(columns={'AIRLINE_x': 'IATA_CODE','AIRLINE_y': 'AIRLINE'}, inplace=True)
flights_data.head()

,IATA_CODE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,...,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,AIRLINE
0,AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,...,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Alaska Airlines Inc.
1,AA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,...,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.
2,US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,...,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,US Airways Inc.
3,AA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,...,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.
4,AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,...,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Alaska Airlines Inc.


In [5]:
flights_data['SPEED'] = (60*flights_data['DISTANCE'])/(0.62137119*flights_data['AIR_TIME'])
flights_data.head()

,IATA_CODE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,...,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,AIRLINE,SPEED
0,AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Alaska Airlines Inc.,827.336137
1,AA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.,855.461186
2,US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,US Airways Inc.,833.470790
3,AA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.,876.531084
4,AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Alaska Airlines Inc.,702.612097


In [6]:
d = {'IATA_CODE': flights_data.IATA_CODE, 'AIRLINE': flights_data.AIRLINE, 'SPEED': flights_data.SPEED}
df = pd.DataFrame(data=d)
df.to_csv("speed.csv", encoding='utf-8', index=False)

sns.set_context("notebook", font_scale=2.5)
sns.set(style="ticks", palette="muted", color_codes=True)
ax = sns.violinplot(x="SPEED", y="AIRLINE", data=flights_data)
sns.despine(trim=True)
plt.show()

In [20]:
codes = []
values = []
for airline in airlines_data.IATA_CODE:
    airline_describe = flights_data.SPEED[flights_data.IATA_CODE == airline].describe()
    for i in range(5):
        codes.append(airline)
    values.append(airline_describe['min'])
    values.append(airline_describe['25%'])
    values.append(airline_describe['50%'])
    values.append(airline_describe['75%'])
    values.append(airline_describe['max'])

d = {'IATA_CODE': codes, 'SPEED': values}
df = pd.DataFrame(data=d)
df.to_csv("speed.csv", encoding='utf-8', index=False)

In [8]:
aa = flights_data.SPEED[flights_data.IATA_CODE == 'AA'].describe()

count    194223.000000
mean        661.614993
std         111.044888
min         151.964286
25%         598.120225
50%         667.045343
75%         731.101991
max        1126.669552
Name: SPEED, dtype: float64

In [15]:
aa['mean']

688.28317254778403

In [21]:
aa

count    712935.000000
mean        688.283173
std         104.144824
min         128.090645
25%         631.585413
50%         701.718078
75%         757.629640
max        1266.190332
Name: SPEED, dtype: float64